# Wages vs. Rent — SF Bay Area

**Question.** Did wages keep up with rent in the San Francisco–Oakland–Hayward metro, and what happened to affordability (real wages and wage-to-rent ratio) over time?

**What you'll see.**
- Latest **FRED** pulls for wages and prices
- Indexed levels (2015=100), **real wages**, **wage-to-rent ratio**
- **YoY** growth paths and **CAGR** by subperiods
- A simple growth regression with **robust standard errors**


In [1]:
import sys, subprocess
def pip_install(pkg):
    subprocess.check_call([sys.executable, "-m", "pip", "install", pkg])
for p in ["pandas", "numpy", "statsmodels", "pandas_datareader", "plotly"]:
    try:
        __import__(p.split("==")[0])
    except Exception:
        pip_install(p)

import pandas as pd, numpy as np, plotly.graph_objects as go
from pandas_datareader import data as web
import statsmodels.api as sm
from datetime import datetime
import math, pathlib, textwrap


In [2]:
CONFIG = {
    "out_dir": "./",
    "index_base": "2015-01-01",          # Index base date (for 2015=100)
    "sf": {
        "name": "SF–Oakland–Hayward, CA",
        "wages_sa": "ENUC418640510SA",   # Seasonally adjusted (preferred)
        "wages_nsa": "ENUC418640510",    # Not seasonally adjusted
        "cpi_all": "CUUSA422SA0",        # CPI-U: All items, SF Bay (NSA)
        "cpi_rent": "CUUSA422SEHA",      # CPI-U: Rent of Primary Residence, SF Bay (NSA)
    },
    "other": {
        "enabled": False,
        "name": "Los Angeles–Long Beach–Anaheim, CA",
        "wages_sa": "ENUC310840510SA",   # LA wages (SA)
        "wages_nsa": "ENUC310840510",    # LA wages (NSA)
        "cpi_all": None,                 # e.g., "CUUSA421SA0" (often discontinued)
        "cpi_rent": None,                # e.g., "CUUSA421SEHA" (often discontinued)
    }
}


In [3]:
def fred_series(series, start='1990-01-01'):
    """Fetch a FRED series, return a DataFrame with 'date' and 'value'."""
    s = web.DataReader(series, "fred", start=start)
    s = s.rename(columns={series: "value"}).reset_index().rename(columns={"DATE":"date"})
    return s

def to_quarterly_avg(df, date_col="date", val_col="value"):
    d = df.copy()
    d[date_col] = pd.to_datetime(d[date_col])
    d = d.set_index(date_col).resample("QS").mean().reset_index()
    d = d.rename(columns={val_col: val_col})
    return d

def index_series(df, base_date, val_col="value", out_col="index_2015=100"):
    base_date = pd.to_datetime(base_date)
    base_val = df.loc[df["date"]==base_date, val_col]
    if base_val.empty:
        base_year = base_date.year
        base_q = df[df["date"].dt.year==base_year].iloc[0][val_col]
        base_val = pd.Series([base_q])
    base_val = float(base_val.iloc[0])
    r = df.copy()
    r[out_col] = r[val_col] / base_val * 100.0
    return r

def yoy_pct(df, val_col, periods=4):
    out = df.copy()
    out[f"{val_col}_yoy_pct"] = out[val_col].pct_change(periods)*100
    return out

def cagr(start_val, end_val, n_years):
    if start_val<=0 or end_val<=0 or n_years<=0:
        return np.nan
    return (end_val/start_val)**(1.0/n_years)-1.0

def subperiod_cagrs(df, date_col, val_col, cuts):
    """Compute CAGR between consecutive cut points in 'cuts' (list of dates)."""
    df = df.copy()
    df[date_col] = pd.to_datetime(df[date_col])
    results = []
    for i in range(len(cuts)-1):
        a, b = pd.to_datetime(cuts[i]), pd.to_datetime(cuts[i+1])
        seg = df[(df[date_col]>=a) & (df[date_col]<=b)]
        if seg.empty:
            results.append((a, b, np.nan))
            continue
        start = float(seg.iloc[0][val_col])
        end   = float(seg.iloc[-1][val_col])
        years = (b - a).days/365.25
        results.append((a, b, cagr(start, end, years)))
    return pd.DataFrame(results, columns=["start","end","cagr"])


In [4]:
BASE = CONFIG["index_base"]
sf_w_sa = fred_series(CONFIG["sf"]["wages_sa"])
sf_w_sa_q = to_quarterly_avg(sf_w_sa, "date", "value").rename(columns={"value":"wage_sa"})
sf_cpi_all = fred_series(CONFIG["sf"]["cpi_all"])
sf_cpi_rent = fred_series(CONFIG["sf"]["cpi_rent"])

sf_cpi_all_q = to_quarterly_avg(sf_cpi_all).rename(columns={"value":"cpi_all"})
sf_cpi_rent_q = to_quarterly_avg(sf_cpi_rent).rename(columns={"value":"cpi_rent"})

sf = sf_w_sa_q.merge(sf_cpi_all_q, on="date", how="inner").merge(sf_cpi_rent_q, on="date", how="inner")

sf["real_wage"] = sf["wage_sa"] / sf["cpi_all"]

sf["wage_to_rent"] = sf["wage_sa"] / sf["cpi_rent"]

sf = index_series(sf, BASE, val_col="wage_sa", out_col="wage_index_2015=100")
sf = index_series(sf, BASE, val_col="cpi_rent", out_col="rent_index_2015=100")
sf = index_series(sf, BASE, val_col="cpi_all", out_col="cpi_all_index_2015=100")
sf = index_series(sf, BASE, val_col="real_wage", out_col="real_wage_index_2015=100")
sf = index_series(sf, BASE, val_col="wage_to_rent", out_col="wage_to_rent_index_2015=100")

sf = yoy_pct(sf, "wage_sa", periods=4)
sf = yoy_pct(sf, "cpi_rent", periods=4)

sf.tail(6)


/tmp/ipython-input-684724335.py:28: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  out[f"{val_col}_yoy_pct"] = out[val_col].pct_change(periods)*100


,date,wage_sa,cpi_all,cpi_rent,real_wage,wage_to_rent,wage_index_2015=100,rent_index_2015=100,cpi_all_index_2015=100,real_wage_index_2015=100,wage_to_rent_index_2015=100,wage_sa_yoy_pct,cpi_rent_yoy_pct
134,2023-07-01,2551.976282,340.411,502.741,7.496750,5.076125,162.375327,137.782181,132.598560,122.456328,117.849294,2.682705,4.754294
135,2023-10-01,2570.574144,NaN,NaN,NaN,NaN,163.558659,NaN,NaN,NaN,NaN,8.113535,4.754294
136,2024-01-01,2679.643363,347.857,504.951,7.703290,5.306739,170.498437,138.387858,135.498962,125.830069,123.203321,7.653804,2.101679
137,2024-04-01,2640.875059,NaN,NaN,NaN,NaN,168.031715,NaN,NaN,NaN,NaN,5.146922,2.101679
138,2024-07-01,2655.638867,348.977,511.267,7.609782,5.194231,168.971096,140.118833,135.935230,124.302653,120.591281,4.062051,1.695903
139,2024-10-01,2725.645910,NaN,NaN,NaN,NaN,173.425455,NaN,NaN,NaN,NaN,6.032573,1.695903


In [5]:
def to_quarter_period(df, value_col="value"):
    d = df.copy()
    d["quarter"] = pd.to_datetime(d["date"]).dt.to_period("Q")
    return (d.groupby("quarter")[value_col]
             .mean()
             .to_frame())

w_q   = to_quarter_period(sf_w_sa,     "value").rename(columns={"value":"wage_sa"})
cpi_q = to_quarter_period(sf_cpi_all,  "value").rename(columns={"value":"cpi_all"})
rnt_q = to_quarter_period(sf_cpi_rent, "value").rename(columns={"value":"cpi_rent"})

sf = w_q.join(cpi_q, how="inner").join(rnt_q, how="inner").dropna().reset_index()
sf["date"] = sf["quarter"].dt.to_timestamp("Q")  # quarter end timestamp

BASE = pd.to_datetime(CONFIG["index_base"])
sf["real_wage"]     = sf["wage_sa"] / sf["cpi_all"]
sf["wage_to_rent"]  = sf["wage_sa"] / sf["cpi_rent"]

def idx(df, col, base=BASE):
    base_row = df.loc[df["date"]==base, col]
    if base_row.empty:
        base_row = pd.Series([df[col].iloc[0]])
    return df[col] / float(base_row.iloc[0]) * 100

sf["wage_index_2015=100"]         = idx(sf, "wage_sa")
sf["rent_index_2015=100"]         = idx(sf, "cpi_rent")
sf["cpi_all_index_2015=100"]      = idx(sf, "cpi_all")
sf["real_wage_index_2015=100"]    = idx(sf, "real_wage")
sf["wage_to_rent_index_2015=100"] = idx(sf, "wage_to_rent")

sf["wage_sa_yoy_pct"] = sf["wage_sa"].pct_change(4) * 100
sf["cpi_rent_yoy_pct"] = sf["cpi_rent"].pct_change(4) * 100

print("Rows after alignment:", len(sf))
sf.tail(4)[["date","wage_sa","cpi_rent","cpi_all"]]


Rows after alignment: 70


,date,wage_sa,cpi_rent,cpi_all
66,2023-03-31,2489.130206,494.557,337.689
67,2023-09-30,2551.976282,502.741,340.411
68,2024-03-31,2679.643363,504.951,347.857
69,2024-09-30,2655.638867,511.267,348.977


In [6]:
import plotly.graph_objects as go

def linefig(df, ycols, title, ytitle="Index (2015=100)"):
    fig = go.Figure()
    for col in ycols:
        fig.add_trace(go.Scatter(x=df["date"], y=df[col], mode="lines", name=col))
    fig.update_layout(title=title, xaxis_title="Quarter", yaxis_title=ytitle, template="plotly_white")
    fig.show()

linefig(sf, ["wage_index_2015=100","rent_index_2015=100"],
        "SF Bay: Wages vs. Rent (2015=100)")

linefig(sf, ["real_wage_index_2015=100","rent_index_2015=100"],
        "SF Bay: Real Wages vs. Rent (2015=100)")

linefig(sf, ["wage_to_rent_index_2015=100"],
        "SF Bay: Wage-to-Rent Ratio (2015=100)", ytitle="Index (2015=100)")

fig = go.Figure()
fig.add_trace(go.Scatter(x=sf["date"], y=sf["wage_sa_yoy_pct"], mode="lines", name="Wages YoY %"))
fig.add_trace(go.Scatter(x=sf["date"], y=sf["cpi_rent_yoy_pct"], mode="lines", name="Rent CPI YoY %"))
fig.update_layout(title="SF Bay: YoY % — Wages vs. Rent CPI", xaxis_title="Quarter", yaxis_title="Percent", template="plotly_white")
fig.add_hline(y=0, line_width=1, line_dash="dot")
fig.show()


In [7]:
cuts = ["2015-01-01","2018-01-01","2020-01-01","2022-01-01", pd.to_datetime(sf["date"].max()).strftime("%Y-%m-%d")]
cagr_wage = subperiod_cagrs(sf, "date", "wage_index_2015=100", cuts)
cagr_rent = subperiod_cagrs(sf, "date", "rent_index_2015=100", cuts)
cagr_real = subperiod_cagrs(sf, "date", "real_wage_index_2015=100", cuts)
cagr_ratio = subperiod_cagrs(sf, "date", "wage_to_rent_index_2015=100", cuts)

def pretty(df, label):
    d = df.copy()
    d[label] = (d["cagr"]*100).round(2)
    return d[["start","end",label]]

tbl = pretty(cagr_wage, "Wage CAGR %").merge(
      pretty(cagr_rent, "Rent CPI CAGR %"), on=["start","end"]).merge(
      pretty(cagr_real, "Real Wage CAGR %"), on=["start","end"]).merge(
      pretty(cagr_ratio, "Wage-to-Rent CAGR %"), on=["start","end"])

tbl


,start,end,Wage CAGR %,Rent CPI CAGR %,Real Wage CAGR %,Wage-to-Rent CAGR %
0,2015-01-01,2018-01-01,3.95,4.92,1.41,-0.92
1,2018-01-01,2020-01-01,3.35,2.54,0.85,0.80
2,2020-01-01,2022-01-01,15.58,0.05,13.03,15.52
3,2022-01-01,2024-09-30,4.24,3.07,1.40,1.14


In [8]:
reg = sf[["date","wage_sa","cpi_rent"]].copy()
reg["dlog_wage"] = np.log(reg["wage_sa"]).diff()
reg["dlog_rent"] = np.log(reg["cpi_rent"]).diff()
reg = reg.dropna()
print("Obs for regression:", len(reg))

if len(reg) >= 8:  # need a few quarters to estimate
    X = sm.add_constant(reg["dlog_rent"])
    y = reg["dlog_wage"]
    model = sm.OLS(y, X).fit(cov_type="HC1")
    print(model.summary())
else:
    print("Not enough overlapping quarters yet. Showing correlation instead:")
    print(reg[["dlog_wage","dlog_rent"]].corr())


Obs for regression: 69
                            OLS Regression Results                            
Dep. Variable:              dlog_wage   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                 -0.015
Method:                 Least Squares   F-statistic:                  0.001107
Date:                Wed, 27 Aug 2025   Prob (F-statistic):              0.974
Time:                        02:02:56   Log-Likelihood:                 128.96
No. Observations:                  69   AIC:                            -253.9
Df Residuals:                      67   BIC:                            -249.5
Df Model:                           1                                         
Covariance Type:                  HC1                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0229      0.

In [9]:
from pathlib import Path
Path("data").mkdir(parents=True, exist_ok=True)
Path("figures").mkdir(parents=True, exist_ok=True)  # optional


In [10]:
out_dir = pathlib.Path(CONFIG["out_dir"])
out_dir.mkdir(parents=True, exist_ok=True)

sf.to_csv(out_dir / "data/sf_quarterly_wage_rent.csv", index=False)

print("Saved:", out_dir / "data/sf_quarterly_wage_rent.csv")


Saved: data/sf_quarterly_wage_rent.csv


In [11]:
import pandas as pd, numpy as np

base_date = pd.to_datetime(CONFIG["index_base"])
base_idx = sf.index[sf["date"]==base_date]
b = sf.iloc[base_idx[0]] if len(base_idx) else sf.iloc[0]
e = sf.iloc[-1]

def pct(a,b): return (a/b - 1)*100

out = {
  "Period": f"{sf['date'].iloc[0].date()} → {sf['date'].iloc[-1].date()}",
  "Wages %Δ": round(pct(e["wage_sa"], b["wage_sa"]), 1),
  "Rent CPI %Δ": round(pct(e["cpi_rent"], b["cpi_rent"]), 1),
  "Real wages %Δ": round(pct(e["real_wage"], b["real_wage"]), 1),
  "Wage-to-rent %Δ": round(pct(e["wage_to_rent"], b["wage_to_rent"]), 1),
}
for k,v in out.items(): print(f"{k}: {v}")


Period: 1990-03-31 → 2024-09-30
Wages %Δ: 395.1
Rent CPI %Δ: 242.0
Real wages %Δ: 84.6
Wage-to-rent %Δ: 44.8


In [12]:
from pathlib import Path
Path("artifacts").mkdir(exist_ok=True)

base_date = pd.to_datetime(CONFIG["index_base"])
brow = sf.loc[sf["date"].eq(base_date)]
b = sf.iloc[0] if brow.empty else brow.iloc[0]
e = sf.iloc[-1]
pct = lambda a,b: (a/b - 1)*100

period = f"{sf['date'].iloc[0].date()} → {sf['date'].iloc[-1].date()}"
w  = round(pct(e["wage_sa"],     b["wage_sa"]),     1)
r  = round(pct(e["cpi_rent"],    b["cpi_rent"]),    1)
rw = round(pct(e["real_wage"],   b["real_wage"]),   1)
wr = round(pct(e["wage_to_rent"], b["wage_to_rent"]), 1)

brief = f"""# Bay Area Wages vs. Rent — Summary

**Question.** Did wages keep up with rent in the San Francisco–Oakland–Hayward metro, and what happened to affordability?

**Data.** BLS/QCEW wages (FRED {CONFIG['sf']['wages_sa']}); CPI-U All Items (FRED {CONFIG['sf']['cpi_all']}); CPI-U Rent of Primary Residence (FRED {CONFIG['sf']['cpi_rent']}). Monthly CPI averaged to quarters; wages are quarterly; base index = {pd.to_datetime(CONFIG['index_base']).date()}.

**Method.** Align to the same quarters; compute indices (2015=100), real wages (deflated by local CPI), wage-to-rent ratio, YoY, and CAGRs. Simple Δlog regression of wages on rent with robust SE.

**Headline findings ({period}).**
- Wages: **{w}%**
- Rent CPI: **{r}%**
- Real wages: **{rw}%**
- Wage-to-rent ratio: **{wr}%**

**Interpretation.** If wages rose faster than rent CPI, affordability pressures eased; if rent CPI outpaced wages, they tightened. The wage-to-rent index summarizes affordability in one line (↑ is better).

**Limits.** CPI-Rent is a price index for renters (not listing medians like Zillow ZORI). QCEW wages include bonuses/stock; Bay Area quarters can spike. Descriptive analysis; correlation ≠ causation.

**Figures.** See `figures/sf_wage_vs_rent.png`, `figures/sf_real_wage_vs_rent.png`, `figures/sf_yoy.png`.
"""
open("artifacts/brief.md","w").write(brief)
print("Wrote artifacts/brief.md")


Wrote artifacts/brief.md


In [13]:
!apt-get -qq install -y wkhtmltopdf
!pip -q install pdfkit markdown

from pathlib import Path
import markdown, pdfkit

Path("artifacts").mkdir(exist_ok=True)
html_body = markdown.markdown(Path("artifacts/brief.md").read_text())

css = """
<style>
body{font-family:-apple-system,BlinkMacSystemFont,"Segoe UI",Roboto,"Helvetica Neue",Arial,sans-serif;
      margin:1in;font-size:12pt;line-height:1.4;}
h1{font-size:20pt;margin:0 0 .15in 0;}
h2{font-size:14pt;margin:.3in 0 .1in 0;}
</style>
"""
html = f"<!doctype html><html><head><meta charset='utf-8'>{css}</head><body>{html_body}</body></html>"
Path("artifacts/brief.html").write_text(html, encoding="utf-8")

config = pdfkit.configuration(wkhtmltopdf="/usr/bin/wkhtmltopdf")
pdfkit.from_file("artifacts/brief.html", "artifacts/brief.pdf", configuration=config)

from google.colab import files
files.download("artifacts/brief.pdf")


Extracting templates from packages: 100%
Selecting previously unselected package libavahi-core7:amd64.
(Reading database ... 126371 files and directories currently installed.)
Preparing to unpack .../0-libavahi-core7_0.8-5ubuntu5.2_amd64.deb ...
Unpacking libavahi-core7:amd64 (0.8-5ubuntu5.2) ...
Selecting previously unselected package libdaemon0:amd64.
Preparing to unpack .../1-libdaemon0_0.14-7.1ubuntu3_amd64.deb ...
Unpacking libdaemon0:amd64 (0.14-7.1ubuntu3) ...
Selecting previously unselected package avahi-daemon.
Preparing to unpack .../2-avahi-daemon_0.8-5ubuntu5.2_amd64.deb ...
Unpacking avahi-daemon (0.8-5ubuntu5.2) ...
Selecting previously unselected package libqt5core5a:amd64.
Preparing to unpack .../3-libqt5core5a_5.15.3+dfsg-2ubuntu0.2_amd64.deb ...
Unpacking libqt5core5a:amd64 (5.15.3+dfsg-2ubuntu0.2) ...
Selecting previously unselected package libevdev2:amd64.
Preparing to unpack .../4-libevdev2_1.12.1+dfsg-1_amd64.deb ...
Unpacking libevdev2:amd64 (1.12.1+dfsg-1) ...
S

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [14]:
!zip -r artifacts.zip figures data artifacts -q
from google.colab import files
files.download("artifacts.zip")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [15]:
piq = f"""I built a reproducible analysis of Bay Area affordability using BLS/FRED data.
I aligned quarterly wages with local CPI and rent CPI, then tracked real wages and a wage-to-rent index from 2015 to present.
Over {period}, wages changed ~{w}%, rent CPI ~{r}%, real wages ~{rw}%, and the wage-to-rent index ~{wr}%.
I documented the pipeline in a Colab notebook and generated publication-quality figures.
This taught me time-series cleaning, indexing/deflation, and results communication—tools I’m excited to bring to upper-division economics research."""
print(piq)

resume = [
  "Built Python/Colab time-series pipeline (pandas/statsmodels/Plotly) analyzing SF Bay wages vs. rent; reproducible FRED/BLS workflow, CSV + charts.",
  f"Quantified affordability via real wages and wage-to-rent index; {period} headline deltas — wages {w}%, rent CPI {r}%, real wages {rw}%."
]
for b in resume: print("•", b)


I built a reproducible analysis of Bay Area affordability using BLS/FRED data.
I aligned quarterly wages with local CPI and rent CPI, then tracked real wages and a wage-to-rent index from 2015 to present.
Over 1990-03-31 → 2024-09-30, wages changed ~395.1%, rent CPI ~242.0%, real wages ~84.6%, and the wage-to-rent index ~44.8%.
I documented the pipeline in a Colab notebook and generated publication-quality figures.
This taught me time-series cleaning, indexing/deflation, and results communication—tools I’m excited to bring to upper-division economics research.
• Built Python/Colab time-series pipeline (pandas/statsmodels/Plotly) analyzing SF Bay wages vs. rent; reproducible FRED/BLS workflow, CSV + charts.
• Quantified affordability via real wages and wage-to-rent index; 1990-03-31 → 2024-09-30 headline deltas — wages 395.1%, rent CPI 242.0%, real wages 84.6%.


In [16]:
from pathlib import Path
import pandas as pd, numpy as np

base_date = pd.to_datetime(CONFIG["index_base"])
brow = sf.loc[sf["date"].eq(base_date)]
b = sf.iloc[0] if brow.empty else brow.iloc[0]
e = sf.iloc[-1]
pct = lambda a,b: (a/b - 1)*100

period = f"{sf['date'].iloc[0].date()} → {sf['date'].iloc[-1].date()}"
w  = round(pct(e["wage_sa"],     b["wage_sa"]),     1)
r  = round(pct(e["cpi_rent"],    b["cpi_rent"]),    1)
rw = round(pct(e["real_wage"],   b["real_wage"]),   1)
wr = round(pct(e["wage_to_rent"], b["wage_to_rent"]), 1)

Path("figures").mkdir(exist_ok=True)
Path("data").mkdir(exist_ok=True)
Path("artifacts").mkdir(exist_ok=True)

readme = f"""# Bay Area Wages vs. Rent (SF–Oakland–Hayward)

**Question.** Did wages keep up with rent in the SF Bay Area, and what happened to affordability (real wages and wage-to-rent ratio)?

**Headline ({period})**
- **Wages:** {w}%  |  **Rent CPI:** {r}%  |  **Real wages:** {rw}%  |  **Wage-to-rent index:** {wr}%

**Figures**
![Wages vs Rent (2015=100)](figures/sf_wage_vs_rent.png)
![Real Wages vs Rent (2015=100)](figures/sf_real_wage_vs_rent.png)
![YoY % — Wages vs Rent CPI](figures/sf_yoy.png)

**Data & Methods**
- Wages: BLS QCEW, FRED series `{CONFIG['sf']['wages_sa']}` (quarterly, SA).
- Prices: CPI-U (All Items `{CONFIG['sf']['cpi_all']}`) and Rent of Primary Residence `{CONFIG['sf']['cpi_rent']}` (monthly → quarterly average).
- Both series aligned to the **same quarters**; indices normalized to **{pd.to_datetime(CONFIG['index_base']).date()} = 100**.
- Affordability proxies: **real wages** (deflated by local CPI) and **wage-to-rent ratio**.

**Reproducibility**
- Notebook: `analysis.ipynb` (Colab-ready).
- Key dataset: `data/sf_quarterly_wage_rent.csv`.
- To re-run: `pip install pandas numpy statsmodels pandas_datareader plotly` and execute the notebook.

**Notes / Limits**
- CPI-Rent is a price index (not listing medians); QCEW wages can spike with bonuses/stock; analysis is descriptive, not causal.

**License**: MIT (optional).
"""

Path("README.md").write_text(readme, encoding="utf-8")

!zip -r PROOF.zip README.md figures data artifacts/brief.pdf -q

from google.colab import files
files.download("PROOF.zip")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Interpretation
- If **wage_index** rose faster than **rent_index**, affordability pressure **eased**; if the opposite, it **tightened**.
- The **wage-to-rent** index summarizes affordability in one line (↑ is better).
- **Real wages** (deflated by local CPI) show purchasing power independent of rent-specific inflation.

## Limits and cautions
- CPI rent is a **price index** across renters; it differs from listing medians (e.g., Zillow ZORI).
- Wages (QCEW) include bonuses/stock — Bay Area quarters can spike.
- This is **descriptive**; correlation ≠ causation.

## Sources
- FRED, St. Louis Fed: `ENUC418640510SA` (wages), `CUUSA422SA0` (CPI All Items), `CUUSA422SEHA` (CPI Rent).  
- BLS QCEW; BLS CPI-U (local area).


## Figure Export Settings

- Forces **yearly ticks** (clean `%Y` labels) so the x‑axis isn’t cramped or weird.  
- Uses consistent sizing and DPI so PNGs look sharp in a slide or PDF.  
- Gives you a **Plotly** export and a **Matplotlib fallback** (no Chrome/kaleido needed).

Place this **below your “Interactive charts (Plotly)” section** and **run after the cell that builds `sf`**.


In [17]:
from pathlib import Path
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

Path("figures").mkdir(exist_ok=True)

def mpl_style(ax, title, ylabel):
    ax.set_title(title, fontsize=16, weight="bold")
    ax.set_ylabel(ylabel, fontsize=12)
    ax.set_xlabel("Quarter", fontsize=12)
    ax.grid(True, which="both", alpha=0.25)
    ax.xaxis.set_major_locator(mdates.YearLocator(base=1))
    ax.xaxis.set_major_formatter(mdates.DateFormatter("%Y"))

fig, ax = plt.subplots(figsize=(12, 6.75))
ax.plot(sf["date"], sf["wage_index_2015=100"], label="Wages (2015=100)", linewidth=2)
ax.plot(sf["date"], sf["rent_index_2015=100"],  label="Rent CPI (2015=100)", linewidth=2)
mpl_style(ax, "SF Bay: Wages vs Rent (2015=100)", "Index (2015=100)")
ax.legend(frameon=False, loc="upper left")
fig.tight_layout(); fig.savefig("figures/sf_wage_vs_rent.png", dpi=300); plt.close(fig)

fig, ax = plt.subplots(figsize=(12, 6.75))
ax.plot(sf["date"], sf["real_wage_index_2015=100"], label="Real wages (2015=100)", linewidth=2)
ax.plot(sf["date"], sf["rent_index_2015=100"],      label="Rent CPI (2015=100)", linewidth=2)
mpl_style(ax, "SF Bay: Real Wages vs Rent (2015=100)", "Index (2015=100)")
ax.legend(frameon=False, loc="upper left")
fig.tight_layout(); fig.savefig("figures/sf_real_wage_vs_rent.png", dpi=300); plt.close(fig)

fig, ax = plt.subplots(figsize=(12, 6.75))
ax.plot(sf["date"], sf["wage_sa_yoy_pct"], label="Wages YoY %", linewidth=2)
ax.plot(sf["date"], sf["cpi_rent_yoy_pct"], label="Rent CPI YoY %", linewidth=2)
ax.axhline(0, linewidth=1, color="black", alpha=0.5)
mpl_style(ax, "SF Bay: YoY % — Wages vs Rent CPI", "Percent")
ax.legend(frameon=False, loc="upper left")
fig.tight_layout(); fig.savefig("figures/sf_yoy.png", dpi=300); plt.close(fig)

print("Saved PNGs in ./figures")


Saved PNGs in ./figures


In [18]:
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

plt.rcParams.update({
    "figure.dpi": 140,
    "axes.titlesize": 20,
    "axes.titleweight": "bold",
    "axes.labelsize": 12,
    "xtick.labelsize": 11,
    "ytick.labelsize": 11,
})

def style_axis(ax, title, ylabel):
    ax.set_title(title)
    ax.set_ylabel(ylabel)
    ax.set_xlabel("")                      # cleaner
    ax.grid(True, which="major", alpha=0.22)
    ax.xaxis.set_major_locator(mdates.YearLocator(base=1))   # yearly ticks
    ax.xaxis.set_major_formatter(mdates.DateFormatter("%Y")) # show YYYY


In [19]:
from pathlib import Path
Path("figures").mkdir(exist_ok=True)

clip_start = "2010-01-01"
df = sf.loc[sf["date"] >= clip_start].copy()

df["wage_idx_ma4"] = df["wage_index_2015=100"].rolling(4, min_periods=1).mean()
df["rent_idx_ma4"] = df["rent_index_2015=100"].rolling(4, min_periods=1).mean()

fig, ax = plt.subplots(figsize=(13.5, 7.5))
ax.plot(df["date"], df["wage_index_2015=100"], alpha=.35, linewidth=1.5, label="Wages (raw)")
ax.plot(df["date"], df["rent_index_2015=100"], alpha=.35, linewidth=1.5, label="Rent CPI (raw)")
ax.plot(df["date"], df["wage_idx_ma4"], linewidth=2.8, label="Wages (4-qtr avg)")
ax.plot(df["date"], df["rent_idx_ma4"], linewidth=2.8, label="Rent CPI (4-qtr avg)")

style_axis(ax, "SF Bay: Wages vs Rent (2015=100)", "Index (2015=100)")
ax.legend(frameon=False, loc="upper left", ncol=2)
fig.tight_layout()
fig.savefig("figures/sf_wage_vs_rent_clean.png", dpi=300)
plt.close(fig)
print("Saved: figures/sf_wage_vs_rent_clean.png")


Saved: figures/sf_wage_vs_rent_clean.png


In [20]:
df = sf.loc[sf["date"] >= clip_start].copy()
df["real_idx_ma4"] = df["real_wage_index_2015=100"].rolling(4, min_periods=1).mean()
df["rent_idx_ma4"] = df["rent_index_2015=100"].rolling(4, min_periods=1).mean()

fig, ax = plt.subplots(figsize=(13.5, 7.5))
ax.plot(df["date"], df["real_wage_index_2015=100"], alpha=.35, linewidth=1.5, label="Real wages (raw)")
ax.plot(df["date"], df["rent_index_2015=100"],      alpha=.35, linewidth=1.5, label="Rent CPI (raw)")
ax.plot(df["date"], df["real_idx_ma4"], linewidth=2.8, label="Real wages (4-qtr avg)")
ax.plot(df["date"], df["rent_idx_ma4"], linewidth=2.8, label="Rent CPI (4-qtr avg)")

style_axis(ax, "SF Bay: Real Wages vs Rent (2015=100)", "Index (2015=100)")
ax.legend(frameon=False, loc="upper left", ncol=2)
fig.tight_layout()
fig.savefig("figures/sf_real_wage_vs_rent_clean.png", dpi=300)
plt.close(fig)
print("Saved: figures/sf_real_wage_vs_rent_clean.png")


Saved: figures/sf_real_wage_vs_rent_clean.png


In [21]:
df = sf.loc[sf["date"] >= "2015-01-01"].copy()

fig, ax = plt.subplots(figsize=(13.5, 7.5))
ax.plot(df["date"], df["wage_sa_yoy_pct"], linewidth=2.2, label="Wages YoY %")
ax.plot(df["date"], df["cpi_rent_yoy_pct"], linewidth=2.2, label="Rent CPI YoY %")
ax.axhline(0, color="black", linewidth=1, alpha=.6)
style_axis(ax, "SF Bay: YoY % — Wages vs Rent CPI", "Percent")
ax.legend(frameon=False, loc="upper left")
fig.tight_layout()
fig.savefig("figures/sf_yoy_clean.png", dpi=300)
plt.close(fig)
print("Saved: figures/sf_yoy_clean.png")


Saved: figures/sf_yoy_clean.png


In [22]:
from google.colab import files
for f in ["figures/sf_wage_vs_rent_clean.png",
          "figures/sf_real_wage_vs_rent_clean.png",
          "figures/sf_yoy_clean.png"]:
    files.download(f)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>